In [1]:
from ast import List

from torch import tensor
from mytorch import Tensor
import numpy as np

from mytorch.tensor import Tensorable

In [9]:
class AvgPool2d():
    def __init__(self, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0)) -> None:
        super()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def forward(self, x: Tensor) -> Tensor:

        def zero_padding(x, padding_size=(1,1)):
            data= np.pad(x, padding_size, mode='constant', constant_values=0)
            return data
        
        "TODO: implement forward pass"
        batch_size, channel_numbers, H, W = x.shape
        H_out = (H + 2 * self.padding[0] - self.kernel_size[0]) // self.stride[0] + 1
        W_out = (W + 2 * self.padding[1] - self.kernel_size[1]) // self.stride[1] + 1

        if self.padding is not None and self.padding!=(0,0):
            batch_data= []
            for data in x.data:
                channel_data = []
                for channel in data:
                    channel_data.append(zero_padding(channel))
                batch_data.append(channel_data)
            data= batch_data
        else:
            data= x.data

        output_data = np.zeros((batch_size, channel_numbers, H_out, W_out))
        for n in range(batch_size):
            for c in range(channel_numbers):
                for h in range(H_out):
                    for w in range(W_out):
                        output_data[n, c, h, w] = np.mean(data[n, c, h * self.stride[0] + self.padding[0] : h * self.stride[0] + self.padding[0] + self.kernel_size[0],
                                                        w * self.stride[1] + self.padding[1] : w * self.stride[1] + self.padding[1] + self.kernel_size[1]])
        
        return Tensor(output_data, x.requires_grad, x.depends_on)
    
    def __str__(self) -> str:
        return "avg pool 2d - kernel: {}, stride: {}, padding: {}".format(self.kernel_size, self.stride, self.padding)
    
a= Tensor([[[[1,3,1],[3,1,2]], [[1,3,1],[3,1,2]]], [[[2,2,2],[2,2,2]], [[1,3,1],[3,1,2]]]])
print(a)
my_average_pool= AvgPool2d(kernel_size=(2,2))
print(my_average_pool.forward(a))

Tensor([[[[1 3 1]
   [3 1 2]]

  [[1 3 1]
   [3 1 2]]]


 [[[2 2 2]
   [2 2 2]]

  [[1 3 1]
   [3 1 2]]]], requires_grad=False)
Tensor([[[[2.   1.75]]

  [[2.   1.75]]]


 [[[2.   2.  ]]

  [[2.   1.75]]]], requires_grad=False)
